> 此版本适用于 0.0.292 及以下版本的 Langchain 代码。

# 问答

## 用例
此处展示了如何使用 Langchian + 千帆 SDK 完成对特定文档完成获取、切分、转为向量并存储，而后根据你的提问来从文中获取答案。
并且借助 Langsmith 将整个过程可视化展现

## 概览
把一个非结构化的文档转成问答链涉及以下步骤：
1. `Loading`: 首先我们需要加载数据，非结构化的数据可以从多种渠道加载。点击 [LangChain integration hub](https://integrations.langchain.com/) 查看所有 Langchain 支持的 Loader。
每个 Loader 都会返回 Langchian 中的 [`Document`](/docs/components/schema/document) 对象。

2. `Splitting`: [文本切分器](/docs/modules/data_connection/document_transformers/) 把 `Documents` 切分成特定的大小。

3. `Storage`: `Storage` （例如 [vectorstore](/docs/modules/data_connection/vectorstores/)）会将切分的数据储存起来，通常还附带对文本做 [embedding](https://www.pinecone.io/learn/vector-embeddings/) 。

4. `Retrieval`: 用于从 `Storage` 中获取切分的数据，用于后面生成答案。

5. `Generation`: 使用提示词和获取到的数据，搭配 [LLM](/docs/modules/model_io/models/llms/) 来生成回答。

6. `Conversation` (扩展): 添加 [Memory](/docs/modules/memory/) 模块来在你的问答链上实现多轮对话。

![flow.jpeg](img/qa_flow.jpeg)

接下来我们会演示如何一步步构造我们自己的流水线，并且实现我们自己定制化的功能

## Step 0. Prepare

为了能够运行我们的 Demo，首先我们需要下载依赖并且设置环境变量

In [1]:
!pip install langchain==0.0.292
!pip install qianfan==0.0.3

In [11]:
import os

os.environ['QIANFAN_AK'] = "your_api_key"
os.environ['QIANFAN_SK'] = "your_secret_key"

# 此处为 Langsmith 相关功能开关。当且仅当你知道这是做什么用时，可删除注释并设置变量以使用 Langsmith 相关功能
# os.environ['LANGCHAIN_TRACING_V2'] = "true"
# os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
# os.environ['LANGCHAIN_API_KEY'] = "your_langchian_api_key"
# os.environ['LANGCHAIN_PROJECT'] = "your_project_name"

is_chinese = True

if is_chinese:
    WEB_URL = "https://zhuanlan.zhihu.com/p/85289282"
    CUSTOM_PROMPT_TEMPLATE = """
        使用下面的语料来回答本模板最末尾的问题。如果你不知道问题的答案，直接回答 "我不知道"，禁止随意编造答案。
        为了保证答案尽可能简洁，你的回答必须不超过三句话。
        请注意！在每次回答结束之后，你都必须接上 "感谢你的提问" 作为结束语
        以下是一对问题和答案的样例：
            请问：秦始皇的原名是什么
            秦始皇原名嬴政。感谢你的提问。
        
        以下是语料：
        
        {context}
        
        请问：{question}
    """
    QUESTION1 = "明朝的开国皇帝是谁"
    QUESTION2 = "朱元璋是什么时候建立的明朝"
else:
    WEB_URL = "https://lilianweng.github.io/posts/2023-06-23-agent/"
    CUSTOM_PROMPT_TEMPLATE = """
        Use the following pieces of context to answer the question at the end. 
        If you don't know the answer, just say that you don't know, don't try to make up an answer. 
        Use three sentences maximum and keep the answer as concise as possible. 
        Always say "thanks for asking!" at the end of the answer. 
        {context}
        Question: {question}
        Helpful Answer:
    """
    QUESTION1 = "How do agents use Task decomposition?"
    QUESTION2 = "What are the various ways to implemet memory to support it?"

## Step 1. Load

指定一个 `DocumentLoader` 来把你指定的非结构化数据加载成 `Documents`。一个 `Document` 是文字（即 `page_content`）和与之相关的元数据的结合体

In [3]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(WEB_URL)
data = loader.load()

## Step 2. Split

接下来把 `Document` 切分成块，为后续的 embedding 和存入向量数据库做准备。

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 384, chunk_overlap = 0, separators=["\n\n", "\n", " ", "", "。", "，"])
all_splits = text_splitter.split_documents(data)

## Step 3. Store

为了能够查询文档的片段，我们首先需要把它们存储起来，一种比较常见的做法是对文档的内容做 embedding，然后再将 embedding 的向量连同文档一起存入向量数据库中，此处 embedding 用于索引文档。

In [5]:
from langchain.embeddings import QianfanEmbeddingsEndpoint
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=QianfanEmbeddingsEndpoint())

[INFO] [09-18 13:39:08] logging.py:55 [t:8485264192]: trying to refresh access_token
[INFO] [09-18 13:39:08] logging.py:55 [t:8485264192]: sucessfully refresh access_token
[INFO] [09-18 13:39:08] logging.py:55 [t:8485264192]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [09-18 13:39:09] logging.py:55 [t:8485264192]: requesting llm api endpoint: /embeddings/embedding-v1


除了非结构化的文档以外，Langchain 还可以从多种数据源获取数据并将它们存储起来。

![lc.png](img/qa_data_load.png)

## Step 4. Retrieve

我们可以使用 [相似度搜索](https://www.pinecone.io/learn/what-is-similarity-search/) 来从切分的文档内获取数据，获取到的数据会作为最终提交给 LLM 的 prompt 的一部分。

In [6]:
docs = vectorstore.similarity_search_with_relevance_scores(QUESTION1)
[(document.page_content, score) for document, score in docs]

[INFO] [09-18 13:39:10] logging.py:55 [t:8485264192]: requesting llm api endpoint: /embeddings/embedding-v1


[('中国古代史-明朝（1368~1644） -', 0.46182550051839333),
 ('知乎切换模式写文章登录/注册中国古代史-明朝（1368~1644）历史风云看历史，就看历史风云起源元朝末年蒙古统治者残暴，人民受到空前压迫。1351年，元廷征调农民和兵士十几万人治理黄河水患。“治河”和“变钞”导致红巾军起义爆发。郭子兴1351年5月，韩山童、刘福通领导红巾军起义爆发。次年，郭子兴响应，聚众起义，攻占濠州。平民出身的朱元璋受汤和邀请投奔郭子兴，屡立战功，备受郭子兴器重和信任，并娶郭子兴养女马氏为妻。不久，朱元璋离开濠州，发展自己的势力。1355年，刘福通立韩林儿为帝，国号宋，年号龙凤，称小明王，以亳州为都城。郭子兴病故后朱元璋统率郭部，小明王任其为左副元帅。1356年，朱元璋占领集庆路，改名为应天府，并攻下周围战略要地，获取一块立足之地。朱元璋采纳朱升“高筑墙，广积粮，缓称王”的建议。1360年，鄱阳湖水战使陈友谅势力遭到巨大打击。1361',
  0.3850624338011377),
 ('年，小明王封朱元璋为吴国公。1363年，陈友谅败亡。1364年，朱元璋称王，史称“西吴”，与占平江府的东吴王张士诚相区别。1366年小明王、刘福通被廖永忠沉于瓜步江中溺死。1367年，攻下平江，张士诚自尽，后又灭浙江的方国珍。明朝的建立朱元璋1368年正月初四，朱元璋以应天府（南京）为京师，国号大明，年号洪武，朱元璋称帝。后进行北伐和西征，同年以“驱逐胡虏，恢复中华”的口号命徐达、常遇春等将北伐，攻占大都（即北京），元顺帝北逃，彻底结束蒙古在中原的统治，中国再次回归由汉族建立的王朝统治之下。之后朱元璋又相继消灭四川的明升和云南的梁王把匝剌瓦尔密、辽东的纳哈出。又八次派兵深入漠北，大破北元，天下至此初定。洪武之治朱元璋即位后采取轻徭薄赋，恢复社会生产，确立里甲制，配合赋役黄册户籍登记簿册和鱼鳞图册的施行，落实赋税劳役的征收及地方治安的维持。整顿吏治，惩治贪官污',
  0.3646482284032496),
 ('军情，骗取军功封赏，军事打击目标上偏袒努尔哈赤势力，致使明末边患严重，并最终导致清朝入主中国。因此《明史》言：“论者谓：明之亡，实亡于神宗。”在军事方面，万历朝以万历三大征最为功勋卓著，三战皆胜以巩固明朝边疆、守护朝鲜王朝，但也消耗了明朝国库存银

## Step 5. Generate

接下来我们就可以使用我们的大模型（例如文心一言）和 Langchain 的 RetrievalQA 链，来针对这篇文档进行提问并获取我们想要的回答了。

In [7]:
from langchain.chains import RetrievalQA
from langchain.chat_models import QianfanChatEndpoint
from langchain.prompts import PromptTemplate

QA_CHAIN_PROMPT = PromptTemplate.from_template(CUSTOM_PROMPT_TEMPLATE)

llm = QianfanChatEndpoint(streaming=True)
retriever=vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold': 0.0})
                                   
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})
qa_chain({"query": QUESTION1})

[INFO] [09-18 13:39:10] logging.py:55 [t:8485264192]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [09-18 13:39:10] logging.py:55 [t:8485264192]: requesting llm api endpoint: /chat/eb-instant


{'query': '明朝的开国皇帝是谁', 'result': '明朝的开国皇帝是朱元璋。感谢你的提问。'}

注意，此处不光可以传入 `ChatModel` ，也可以传入一个 `LLM` 对象到 `RetrievalQA` 中。并且通过代码我们可以看到，用户可以通过传入额外的命名参数字典来自定义我们所需使用的 prompt 模板

#### 返回源文档

用于 QA 的知识文档也可以通过指定 `return_source_documents=True` 被包含在返回的字典里

In [8]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}, return_source_documents=True)
result = qa_chain({"query": QUESTION1})
len(result['source_documents'])
result['source_documents']

[INFO] [09-18 13:39:11] logging.py:55 [t:8485264192]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [09-18 13:39:12] logging.py:55 [t:8485264192]: requesting llm api endpoint: /chat/eb-instant


[Document(page_content='中国古代史-明朝（1368~1644） -', metadata={'description': '起源元朝末年蒙古统治者残暴，人民受到空前压迫。1351年，元廷征调农民和兵士十几万人治理黄河水患。 “治河”和“变钞”导致红巾军起义爆发。 1351年5月，韩山童、刘福通领导红巾军起义爆发。次年，郭子兴响应，…', 'language': 'zh', 'source': 'https://zhuanlan.zhihu.com/p/85289282', 'title': '中国古代史-明朝（1368~1644） - 知乎'}),
 Document(page_content='知乎切换模式写文章登录/注册中国古代史-明朝（1368~1644）历史风云看历史，就看历史风云起源元朝末年蒙古统治者残暴，人民受到空前压迫。1351年，元廷征调农民和兵士十几万人治理黄河水患。“治河”和“变钞”导致红巾军起义爆发。郭子兴1351年5月，韩山童、刘福通领导红巾军起义爆发。次年，郭子兴响应，聚众起义，攻占濠州。平民出身的朱元璋受汤和邀请投奔郭子兴，屡立战功，备受郭子兴器重和信任，并娶郭子兴养女马氏为妻。不久，朱元璋离开濠州，发展自己的势力。1355年，刘福通立韩林儿为帝，国号宋，年号龙凤，称小明王，以亳州为都城。郭子兴病故后朱元璋统率郭部，小明王任其为左副元帅。1356年，朱元璋占领集庆路，改名为应天府，并攻下周围战略要地，获取一块立足之地。朱元璋采纳朱升“高筑墙，广积粮，缓称王”的建议。1360年，鄱阳湖水战使陈友谅势力遭到巨大打击。1361', metadata={'description': '起源元朝末年蒙古统治者残暴，人民受到空前压迫。1351年，元廷征调农民和兵士十几万人治理黄河水患。 “治河”和“变钞”导致红巾军起义爆发。 1351年5月，韩山童、刘福通领导红巾军起义爆发。次年，郭子兴响应，…', 'language': 'zh', 'source': 'https://zhuanlan.zhihu.com/p/85289282', 'title': '中国古代史-明朝（1368~1644） - 知乎'}),
 Document(page_content='年，小明王封朱元璋为吴国公。1363年，陈友谅败亡。1364年，朱元

## Step 6. Chat

我们还可以加入 `Memory` 模块并替换使用 `ConversationalRetrievalChain` 来实现记忆化的对话式查询。

In [9]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory, combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT})
qa(QUESTION1)

[INFO] [09-18 13:39:13] logging.py:55 [t:8485264192]: requesting llm api endpoint: /chat/eb-instant
[INFO] [09-18 13:39:15] logging.py:55 [t:8485264192]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [09-18 13:39:15] logging.py:55 [t:8485264192]: requesting llm api endpoint: /chat/eb-instant
[INFO] [09-18 13:39:16] logging.py:55 [t:8485264192]: requesting llm api endpoint: /chat/eb-instant


{'question': '明朝的开国皇帝是谁',
 'chat_history': [SystemMessage(content='', additional_kwargs={})],
 'answer': '明朝的开国皇帝是朱元璋。感谢你的提问。'}

In [10]:
qa(QUESTION2)

[INFO] [09-18 13:39:17] logging.py:55 [t:8485264192]: requesting llm api endpoint: /chat/eb-instant
[INFO] [09-18 13:39:18] logging.py:55 [t:8485264192]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [09-18 13:39:19] logging.py:55 [t:8485264192]: requesting llm api endpoint: /chat/eb-instant
[INFO] [09-18 13:39:21] logging.py:55 [t:8485264192]: requesting llm api endpoint: /chat/eb-instant


{'question': '朱元璋是什么时候建立的明朝',
 'chat_history': [SystemMessage(content='明朝的开国皇帝是朱元璋。感谢你的提问。', additional_kwargs={})],
 'answer': '1368年正月初四，朱元璋以应天府（南京）为京师，国号大明，年号洪武，朱元璋称帝。'}